# Generating model datasets

This notebook contains two workflows to generate a model dataset and add layer
information to it:

- Get data (download or read from file)
- Regrid (convert data to match your desired grid)

or

- Create grid (define your desired grid)
- Add data to dataset (add data based on this grid)

In [ ]:
from shapely.geometry import LineString

import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

## Get REGIS data

In [ ]:
extent = [100000, 101000, 400000, 401000]

# downlaod regis
regis = nlmod.read.regis.download_regis(extent)

In [ ]:
f, ax = nlmod.plot.get_map(extent, figsize=5)
nlmod.plot.data_array(regis["top"], edgecolor="k");

## Define some properties of the grid
We choose a resolution of the calculation grid (delr and delc) larger than the resolution of regis (100 meter).

In [ ]:
delr = 200.0
delc = 200.0

polygon = LineString([(extent[0], extent[2]), (extent[1], extent[3])]).buffer(10)
refinement_features = [([polygon], "polygon", 3)]

## Regrid data
create a model dataset from regis and refine.

In [ ]:
ds = nlmod.to_model_ds(regis, delr=delr, delc=delc)
ds = nlmod.grid.refine(ds, model_ws="model_12", refinement_features=refinement_features)

When we plot the model top, we see that all information has a resolution of 200 meter, also in the smaller cells. The original resolution of the regis data is 100 meter however. So information is lost in the process.

In [ ]:
f, ax = nlmod.plot.get_map(extent, figsize=5)
nlmod.plot.data_array(ds["top"], ds=ds, edgecolor="k");

## Add data to existing grid
We can also first create a grid first, and then warp the information from regis to this grid. 

In [ ]:
ds = nlmod.get_ds(extent, delr=delr, delc=delc)
ds = nlmod.grid.refine(ds, model_ws="model_12", refinement_features=refinement_features)
ds = nlmod.grid.update_ds_from_layer_ds(ds, regis, method="average")

When we plot the model top, we now see that in the cells with an equal resolution to that of Regis (or smaller) no information is lost.

In [ ]:
f, ax = nlmod.plot.get_map(extent, figsize=5)
nlmod.plot.data_array(ds["top"], ds=ds, edgecolor="k");